In [1]:
import os
os.environ['XDG_CACHE_HOME'] = '/media/kariters/数据/hugginface/'
os.environ['HF_ENDPOINT']='https://hf-mirror.com'
os.environ['_HF_DEFAULT_ENDPOINT'] = 'https://hf-mirror.com'
print(os.environ['XDG_CACHE_HOME'])

/media/kariters/数据/hugginface/


In [2]:
!pip list | grep vllm
# vllm                                     0.8.5.post1
#transformers                             4.51.3

vllm                                     0.8.5.post1


In [3]:
!pip list | grep transformers

transformers                             4.51.3


In [4]:
from vllm import LLM, SamplingParams
llm = LLM(model="Qwen/Qwen2.5-3B-Instruct", task="generate", trust_remote_code=True)

hf endpoint https://hf-mirror.com
INFO 05-19 18:38:51 [__init__.py:239] Automatically detected platform cuda.
pretrained_model_name_or_path Qwen/Qwen2.5-3B-Instruct
config_dict {'architectures': ['Qwen2ForCausalLM'], 'attention_dropout': 0.0, 'bos_token_id': 151643, 'eos_token_id': 151645, 'hidden_act': 'silu', 'hidden_size': 2048, 'initializer_range': 0.02, 'intermediate_size': 11008, 'max_position_embeddings': 32768, 'max_window_layers': 70, 'model_type': 'qwen2', 'num_attention_heads': 16, 'num_hidden_layers': 36, 'num_key_value_heads': 2, 'rms_norm_eps': 1e-06, 'rope_theta': 1000000.0, 'sliding_window': 32768, 'tie_word_embeddings': True, 'torch_dtype': 'bfloat16', 'transformers_version': '4.43.1', 'use_cache': True, 'use_sliding_window': False, 'vocab_size': 151936, '_commit_hash': 'aa8e72537993ba99e69dfaafa59ed015b17504d1'}
INFO 05-19 18:38:59 [config.py:2003] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 05-19 18:39:01 [core.py:58] Initializing a V1 LLM engin

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 05-19 18:39:05 [loader.py:458] Loading weights took 1.06 seconds
INFO 05-19 18:39:05 [gpu_model_runner.py:1347] Model loading took 5.7916 GiB and 3.632633 seconds
INFO 05-19 18:39:10 [backends.py:420] Using cache directory: /media/kariters/数据/hugginface/vllm/torch_compile_cache/2221e8d4e9/rank_0_0 for vLLM's torch.compile
INFO 05-19 18:39:10 [backends.py:430] Dynamo bytecode transform time: 5.28 s
INFO 05-19 18:39:14 [backends.py:118] Directly load the compiled graph(s) for shape None from the cache, took 3.520 s
INFO 05-19 18:39:15 [monitor.py:33] torch.compile takes 5.28 s in total
INFO 05-19 18:39:16 [kv_cache_utils.py:634] GPU KV cache size: 337,616 tokens
INFO 05-19 18:39:16 [kv_cache_utils.py:637] Maximum concurrency for 32,768 tokens per request: 10.30x
INFO 05-19 18:39:37 [gpu_model_runner.py:1686] Graph capturing finished in 20 secs, took 0.55 GiB
INFO 05-19 18:39:37 [core.py:159] init engine (profile, create kv cache, warmup model) took 31.70 seconds
INFO 05-19 18:39:37 

# tokenize相关

In [5]:
#tokenizer   https://hf-mirror.com/docs/tokenizers/index
tokenizer = llm.get_tokenizer()

# string -> tokens
print(tokenizer('这是测试'))
print(tokenizer(['这是测试', '这']))


{'input_ids': [100346, 81705], 'attention_mask': [1, 1]}
{'input_ids': [[100346, 81705], [43288]], 'attention_mask': [[1, 1], [1]]}


以上内容
* 输入支持string、 [string]， 输出与输入对应

In [6]:
tokenizer.apply_chat_template??

Signature:
tokenizer.apply_chat_template(
    conversation: Union[List[Dict[str, str]], List[List[Dict[str, str]]]],
    tools: Optional[List[Union[Dict, Callable]]] = None,
    documents: Optional[List[Dict[str, str]]] = None,
    chat_template: Optional[str] = None,
    add_generation_prompt: bool = False,
    continue_final_message: bool = False,
    tokenize: bool = True,
    padding: Union[bool, str, transformers.utils.generic.PaddingStrategy] = False,
    truncation: bool = False,
    max_length: Optional[int] = None,
    return_tensors: Union[str, transformers.utils.generic.TensorType, NoneType] = None,
    return_dict: bool = False,
    return_assistant_tokens_mask: bool = False,
    tokenizer_kwargs: Optional[Dict[str, Any]] = None,
    **kwargs,
) -> Union[str, List[int], List[str], List[List[int]], transformers.tokenization_utils_base.BatchEncoding]
Source:   
    def apply_chat_template(
        self,
        conversation: Union[List[Dict[str, str]], List[List[Dict[str, str

In [7]:
# conversation -> template
conversation1 = [{'role': 'user', 'content': '阿巴阿巴'}]
prompts = tokenizer.apply_chat_template(conversation1, tokenize=False, add_generation_prompt=True)
# tokenize：true 输出的是ids， false输出为文本
# add_generation_prompt true 末尾添加 assistant
print('----convertsation template----')
print(prompts)

conversation2 = [{'role':'system', 'content': 'system阿巴'}, {'role': 'user', 'content': '阿巴阿巴'}]
prompts = tokenizer.apply_chat_template(conversation2, tokenize=False, add_generation_prompt=True)
print('----convertsation template----')
print(prompts)
# 观察下后续默认的system prompt
prompts = tokenizer.apply_chat_template(conversation1, tokenize=False, add_generation_prompt=True)
print('----convertsation template----')
print(prompts)

# continue_final_message 用于接着生成的场景
prompts = tokenizer.apply_chat_template(conversation1, tokenize=False, continue_final_message=True)
print('----convertsation template----')
print(prompts)

----convertsation template----
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
阿巴阿巴<|im_end|>
<|im_start|>assistant

----convertsation template----
<|im_start|>system
system阿巴<|im_end|>
<|im_start|>user
阿巴阿巴<|im_end|>
<|im_start|>assistant

----convertsation template----
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
阿巴阿巴<|im_end|>
<|im_start|>assistant

----convertsation template----
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
阿巴阿巴


可以看到
* 每条数据 <|im_start|> role+content <|im_end|>
* conversation中没有system时，输出会包含默认的system prompt
* conversation中有system时, 输出中的system为该system prompt， 该system对后续不会有影响
* continue_final_message = True, 最后一条数据不会有 <|im_end|>, add_generation_prompt与该参数只能有一个为true
  
Q：role里的value如果没和template对齐会不会异常？    
A: 该条数据不会出现在结果中

In [8]:
conversation3 = [{'role': 'user11', 'content': '阿巴阿巴'}]
prompts = tokenizer.apply_chat_template(conversation3, tokenize=False, add_generation_prompt=True)
print('----convertsation template----')
print(prompts)

----convertsation template----
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>assistant



Q:怎么确认支持哪些role?    
A：直接看chat_template

In [9]:
tokenizer.chat_template??

Type:        str
String form:
{%- if tools %}
    {{- '<|im_start|>system\n' }}
    {%- if messages[0]['role'] == 'system' %}
        {{- messages[0]['content'] }}
    {%- else %}
        {{- 'You are Qwen, created by Alibaba Cloud. You are a helpful assistant.' }}
    {%- endif %}
    {{- "\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>" }}
    {%- for tool in tools %}
        {{- "\n" }}
        {{- tool | tojson }}
    {%- endfor %}
    {{- "\n</tools>\n\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\n<tool_call>\n{\"name\": <function-name>, \"arguments\": <args-json-object>}\n</tool_call><|im_end|>\n" }}
{%- else %}
    {%- if messages[0]['role'] == 'system' %}
        {{- '<|im_start|>system\n' + messages[0]['content'] + '<|im_end|>\n' }}
    {%- else %}
        {{- '<|im_start|>system\nYou 

string -> apply_chat_template(tokenize=True) 与  string -> tokenize(apply_chat_template(tokenize=False))结果一致

In [10]:
# 看下 apply_chat_template的ids和 apply_chat_template + tokenize的ids是否一致
conversation1 = [{'role': 'user', 'content': '阿巴阿巴'}]
prompts = tokenizer.apply_chat_template(conversation1, tokenize=False, add_generation_prompt=True)
ids_1 = tokenizer(prompts)['input_ids']
ids_2 = tokenizer.apply_chat_template(conversation1, tokenize=True, add_generation_prompt=True)
print(ids_1)
print(ids_2)


[151644, 8948, 198, 2610, 525, 1207, 16948, 11, 3465, 553, 54364, 14817, 13, 1446, 525, 264, 10950, 17847, 13, 151645, 198, 151644, 872, 198, 99727, 99395, 99727, 99395, 151645, 198, 151644, 77091, 198]
[151644, 8948, 198, 2610, 525, 1207, 16948, 11, 3465, 553, 54364, 14817, 13, 1446, 525, 264, 10950, 17847, 13, 151645, 198, 151644, 872, 198, 99727, 99395, 99727, 99395, 151645, 198, 151644, 77091, 198]


In [11]:
# 看下function call的数据
from datasets import load_dataset
#datasets = load_dataset("glaiveai/glaive-function-calling-v2")
#datasets = load_dataset("Salesforce/xlam-function-calling-60k")
datasets = load_dataset("Jofthomas/hermes-function-calling-thinking-V1")
datasets

[2025-05-19 18:39:37] INFO config.py:54: PyTorch version 2.6.0 available.


hf endpoint https://hf-mirror.com
hf endpoint https://hf-mirror.com
hf endpoint https://hf-mirror.com
hf endpoint https://hf-mirror.com
hf endpoint https://hf-mirror.com
hf endpoint https://hf-mirror.com
hf endpoint https://hf-mirror.com


DatasetDict({
    train: Dataset({
        features: ['conversations'],
        num_rows: 3570
    })
})

In [12]:
import json
from pprint import pprint
def print_sample(data):
    #print(f"---system---\n {data['system']} \n---chat---\n{data['chat']}")
    #print(data['conversations'])
    conversations = data['conversations']
    for msg in conversations:
        pprint(msg)
print_sample(datasets['train'][0])
#print_sample(datasets['train'][1])

{'content': 'You are a function calling AI model. You are provided with '
            'function signatures within <tools></tools> XML tags.You may call '
            "one or more functions to assist with the user query. Don't make "
            'assumptions about what values to plug into functions.Here are the '
            "available tools:<tools> [{'type': 'function', 'function': "
            "{'name': 'get_stock_price', 'description': 'Get the current stock "
            "price of a company', 'parameters': {'type': 'object', "
            "'properties': {'company': {'type': 'string', 'description': 'The "
            "name of the company'}}, 'required': ['company']}}}, {'type': "
            "'function', 'function': {'name': 'get_movie_details', "
            "'description': 'Get details about a movie', 'parameters': "
            "{'type': 'object', 'properties': {'title': {'type': 'string', "
            "'description': 'The title of the movie'}}, 'required': "
            "['tit

In [13]:
# role human->user  model->assistant
def convert_role(data):
    role_map = {'human': 'user', 'model': 'assistant', 'system': 'system', 'tool': 'tool'}
    for d in data['conversations']:
        d['role'] = role_map[d['role']]
        if d['role'] == 'tool':
            d['content'] = d['content'].replace('<tool_response>', '').replace('</tool_response>', '')
    return data
new_datasets = datasets.map(convert_role)
new_datasets['train'][0]

{'conversations': [{'content': "You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags.You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions.Here are the available tools:<tools> [{'type': 'function', 'function': {'name': 'get_stock_price', 'description': 'Get the current stock price of a company', 'parameters': {'type': 'object', 'properties': {'company': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company']}}}, {'type': 'function', 'function': {'name': 'get_movie_details', 'description': 'Get details about a movie', 'parameters': {'type': 'object', 'properties': {'title': {'type': 'string', 'description': 'The title of the movie'}}, 'required': ['title']}}}] </tools>Use the following pydantic model json schema for each tool call you will make: {'title': 'FunctionCall', 'type': 'object', 'properties': {'arguments': {'title'

In [14]:
# tokenize
prompt = tokenizer.apply_chat_template(new_datasets['train'][0]['conversations'], tokenize=False, add_generation_prompt=False)
print(prompt)

<|im_start|>system
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags.You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions.Here are the available tools:<tools> [{'type': 'function', 'function': {'name': 'get_stock_price', 'description': 'Get the current stock price of a company', 'parameters': {'type': 'object', 'properties': {'company': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company']}}}, {'type': 'function', 'function': {'name': 'get_movie_details', 'description': 'Get details about a movie', 'parameters': {'type': 'object', 'properties': {'title': {'type': 'string', 'description': 'The title of the movie'}}, 'required': ['title']}}}] </tools>Use the following pydantic model json schema for each tool call you will make: {'title': 'FunctionCall', 'type': 'object', 'properties': {'arguments': {'title': 'Arguments'

只保留system和user的第一个问题

In [15]:
import re

def extract_tool_call(text):
    pattern = r'<tool_call>(.*?)</tool_call>'
    match = re.search(pattern, text, re.DOTALL)
    res = match.group(1).strip() if match else ""
    res = res.replace("'", '"')
    return res

def trunc_conversations(data):
    data['label'] = data['conversations'][2]['content'] if len( data['conversations']) >= 3 and data['conversations'][2]['role'] == 'assistant' else ''
    data['label'] =  extract_tool_call(data['label'])
    data['conversations'] = data['conversations'][:2]
    return data
trunc_datasets = new_datasets.map(trunc_conversations)
print(trunc_datasets['train'][0]['conversations'])
print(trunc_datasets['train'][0]['label'])

[{'content': "You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags.You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions.Here are the available tools:<tools> [{'type': 'function', 'function': {'name': 'get_stock_price', 'description': 'Get the current stock price of a company', 'parameters': {'type': 'object', 'properties': {'company': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company']}}}, {'type': 'function', 'function': {'name': 'get_movie_details', 'description': 'Get details about a movie', 'parameters': {'type': 'object', 'properties': {'title': {'type': 'string', 'description': 'The title of the movie'}}, 'required': ['title']}}}] </tools>Use the following pydantic model json schema for each tool call you will make: {'title': 'FunctionCall', 'type': 'object', 'properties': {'arguments': {'title': 'Arguments', 'ty

# vllm推理

推理是在做什么：
* 输入string，（string转token ids后）做next token generation
其他任意形式的输入，都会回归到以上形式

In [16]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    #"The capital of France is",
    #"The future of AI is",
]
sampling_params = SamplingParams(temperature=0.8, top_p=0.9, max_tokens=128)

outputs = llm.generate(prompts, sampling_params)

for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt}, Generated text: {generated_text}")

Processed prompts:   0%| | 0/2 [00:00<?, ?it/s, est. speed input: 0.00

Prompt: Hello, my name is, Generated text:  Lisa and I'm a qualified yoga teacher from Kent. I began practicing yoga in 2013 after a series of back injuries which led me to the practice. Yoga has been a life-changing experience for me. It has helped me to be more aware of my body and how it feels, and it has taught me to listen to my body and to respect its needs.
I teach a Hatha yoga style which is a more gentle, slower-paced style of yoga. It is a good introduction to yoga for beginners and a great way to keep practicing, whether you are a complete beginner or have been practicing yoga for years.
I will
Prompt: The president of the United States is, Generated text:  the chief executive of the federal government. The president's role is to take care that the laws are faithfully executed, to receive foreign ambassadors and other public ministers, to take care that the laws be faithfully executed, and to be the commander in chief of the armed forces.
The president's role as the chief ex

## function call示例

In [17]:
sampling_params = SamplingParams(temperature=0.8, top_p=0.9, max_tokens=128)
prompt = tokenizer.apply_chat_template(trunc_datasets['train'][0]['conversations'], tokenize=False, add_generation_prompt=True)
print(prompt)
output = llm.generate([prompt], sampling_params)[0]

prompt = output.prompt
generated_text = output.outputs[0].text
print(f"Generated text: {generated_text}")
print(f"extract from generate: {extract_tool_call(generated_text)}")
print(f"Label: {trunc_datasets['train'][0]['label']}")


<|im_start|>system
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags.You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions.Here are the available tools:<tools> [{'type': 'function', 'function': {'name': 'get_stock_price', 'description': 'Get the current stock price of a company', 'parameters': {'type': 'object', 'properties': {'company': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company']}}}, {'type': 'function', 'function': {'name': 'get_movie_details', 'description': 'Get details about a movie', 'parameters': {'type': 'object', 'properties': {'title': {'type': 'string', 'description': 'The title of the movie'}}, 'required': ['title']}}}] </tools>Use the following pydantic model json schema for each tool call you will make: {'title': 'FunctionCall', 'type': 'object', 'properties': {'arguments': {'title': 'Arguments'

Processed prompts:   0%| | 0/1 [00:00<?, ?it/s, est. speed input: 0.00

Generated text: <tool_call>
{"name": "get_stock_price", "arguments": {"company": "Apple"}}
</tool_call>
extract from generate: {"name": "get_stock_price", "arguments": {"company": "Apple"}}
Label: {"name": "get_stock_price", "arguments": {"company": "Apple"}}


In [18]:
print(json.loads(trunc_datasets['train'][0]['label']))
print(json.loads(extract_tool_call(generated_text)))

{'name': 'get_stock_price', 'arguments': {'company': 'Apple'}}
{'name': 'get_stock_price', 'arguments': {'company': 'Apple'}}


实现一个compare，用来确认generate的内容和label的一致性
* name一致
* arguments一致

In [34]:
def safe_json_load(text):
    try:
        return json.loads(text)
    except:
        return None

def compare(pred, label):
    pred = safe_json_load(pred)
    label = safe_json_load(label)
    if label is None:
        return None
    ret = {'name_right':0, 'arguments_right': 0, 'arguments_num': len(label['arguments'])}
    if pred is None:
        return ret
    ret['name_right'] = int(pred.get('name', '') == label['name'])
    if 'arguments' in pred:
        for arg in label['arguments']:
            ret['arguments_right'] += int(pred['arguments'].get(arg, '') == label['arguments'][arg])
    return ret
compare(extract_tool_call(generated_text), trunc_datasets['train'][0]['label'])


{'name_right': 1, 'arguments_right': 1, 'arguments_num': 1}

In [20]:
def get_pred(conversations):
    '''
    先不用batch，看下速度如何
    '''
    sampling_params = SamplingParams(temperature=0.8, top_p=0.9, max_tokens=512)
    prompt = tokenizer.apply_chat_template(conversations, tokenize=False, add_generation_prompt=True)
    output = llm.generate([prompt], sampling_params)[0]

    prompt = output.prompt
    generated_text = output.outputs[0].text
    return generated_text
get_pred( trunc_datasets['train'][0]['conversations'])

Processed prompts:   0%| | 0/1 [00:00<?, ?it/s, est. speed input: 0.00

'<tool_call>\n{"name": "get_stock_price", "arguments": {"company": "Apple"}}\n</tool_call>'

In [23]:
# 用最后100条数据 评估下准确率
print(len(trunc_datasets['train']))
test_datasets = trunc_datasets['train'][-100:]
print(len(test_datasets['label']))

3570
100


In [25]:
from tqdm import tqdm
eval_datas = []
for conversations, label in tqdm(zip(test_datasets['conversations'], test_datasets['label'])):
    pred = get_pred(conversations)
    eval_datas.append([pred, label])
print(eval_datas[:3])

0it [00:00, ?it/s]

Processed prompts:   0%| | 0/1 [00:00<?, ?it/s, est. speed input: 0.00

1it [00:00,  2.87it/s]

Processed prompts:   0%| | 0/1 [00:00<?, ?it/s, est. speed input: 0.00

2it [00:00,  3.19it/s]

...

Processed prompts:   0%| | 0/1 [00:00<?, ?it/s, est. speed input: 0.00

99it [01:00,  2.76it/s]

Processed prompts:   0%| | 0/1 [00:00<?, ?it/s, est. speed input: 0.00

100it [01:01,  1.64it/s]

[['<tool_call>\n{"name": "search_recipes", "arguments": {"ingredients": ["chicken", "broccoli", "cheese"]}}\n</tool_call>', '{"name": "search_recipes", "arguments": {"ingredients": ["chicken", "broccoli", "cheese"]}}'], ['<tool_call>\n{"name": "search_recipes", "arguments": {"ingredients": ["chicken", "broccoli"]}}\n</tool_call>', '{"name": "search_recipes", "arguments": {"ingredients": ["chicken", "broccoli"]}}'], ['<tool_call>\n{"name": "search_recipes", "arguments": {"ingredients": ["chicken", "broccoli", "cheese"]}}\n</tool_call>', '{"name": "search_recipes", "arguments": {"ingredients": ["chicken", "broccoli", "cheese"]}}']]


In [36]:

sta_dict = {'name_right': 0, 'arguments_right': 0, 'name':0, 'arguments': 0}
for pred, label in eval_datas:
    tool_call_pred = extract_tool_call(pred)
    sta = compare(tool_call_pred, label)
    if sta is None:
        continue
    if sta['arguments_right'] == 0:
        print(label)
    sta_dict['name_right'] += sta['name_right']
    sta_dict['name'] += 1
    sta_dict['arguments_right'] += sta['arguments_right']
    sta_dict['arguments'] += sta['arguments_num']
    
print(sta_dict)
print('name准确率', sta_dict['name_right'] / sta_dict['name'])
print('arg准确率', sta_dict['arguments_right'] / sta_dict['arguments'])

{"name": "search_recipe", "arguments": {"query": "chicken and broccoli"}}
{'name_right': 92, 'arguments_right': 102, 'name': 92, 'arguments': 105}
name准确率 1.0
arg准确率 0.9714285714285714
